# Labor 7


A laboron az osztott és konstans memóriát is használó, rácsozott / csempézett (angolul tiled), kurzuson már ismertetett, képkonvolúció kernelt próbáljuk ki. Egy konstans méretű 5x5-ös konvolúciós maszkkal dolgozunk. A képek mérete  tetszőleges.



Ahhoz, hogy a konvolúciós maszkot a konstans memóriában tároljuk,  ez át kell másolnunk az eszközre. Ha a maszk mutató neve `M `az eszközön, akkor  a kernel definiálásakor használhatjuk a const `float *  __restrict__ M` mutató dekorálást.


```cpp
__global__ void convolution(float *I, const float *__restrict__ M,
                            float *P, int channels, int width,
                            int height)

```

Ez a leírás tájékoztatja a fordítót arról, hogy a maszk tömb tartalma állandó, és csak az `M` mutatóváltozón keresztül érhető el. Ez lehetővé teszi a fordító számára, hogy az adatokat konstans (csak olvasható) memóriába helyezze, és lehetővé teszi az SM hardver számára, hogy futás közben agresszívebb módon gyorsítótárba helyezze (cachelje) a maszkadatokat.

A  [`__restrict__`]( https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#restrict) dekorátor használata csak CC3.5 fölött érhető el, de használata egyszerűbb mint a [`__constant__`]( https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#constant) memóriaterület-specifikátor használata. Továbbá, míg a konstans memóriába másolható maximális adatmennyiség 64 Kb, a `__restrict__`-el megjelölt mutatók esetében nincs egy fix felső határ. 


## Pszeudokód


```
maskWidth := 5
maskRadius := maskWidth/2 # this is integer division, so the result is 2 
for i from 0 to height do
  for j from 0 to width do 
    for k from 0 to channels
      accum := 0
      for y from -maskRadius to maskRadius do
        for x from -maskRadius to maskRadius do 
          xOffset := j + x
          yOffset := i + y
          if xOffset >= 0 && xOffset < width &&
            yOffset >= 0 && yOffset < height then
              imagePixel := I[(yOffset * width + xOffset) * channels + k] 
              maskValue := K[(y+maskRadius)*maskWidth+x+maskRadius]
              accum += imagePixel * maskValue
          end 
        end
      end
        # pixels are in the range of 0 to 1
      P[(i * width + j)*channels + k] = clamp(accum, 0, 1) 
    end
  end
end

def clamp(x, lower, upper)
  return min(max(x, lower), upper)
end

```



# Feladat

Implementáljuk a fentebbi programvázat. Főbb műveletek, részfeladatok:

- memória lefoglalás az eszközön.
- kép másolása az eszközre.
- szálblokk és rács méreteinek, konfigurációjának a meghatározása. 
- CUDA kernel meghívása.
- az eredmény másolása az eszközről a gazdagépre (host).
- memória felszabadítása az eszközön
- 2D "csempézett" konvolúciós kernel implementálása több színcsatornás képekre
- osztott memóriát használva csökkentsük a globális memória hozzáférések számát (a képadatok osztott memóriába való betöltésekor figyelni kell a peremfeltételekre - a maszk részlegesen kilóg a képről).




GPU teszt:

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: ignored

In [2]:
%%writefile conv.cu

#include "libgputk/gputk.h"
#include "limits.h"

static char *base_dir;

char *generateInput(int /*datasetNum*/, char *dirName,
                    gpuTKGenerateParams_t params) {
  char *input_file_name = gpuTKPath_join(dirName, "input0.ppm");
  gpuTKDataset_generate(input_file_name, gpuTKExportKind_ppm, params);
  return input_file_name;
}

char *generateMask(int /*datasetNum*/, char *dirName) {
  // Mask generation parameters
  gpuTKRaw_GenerateParams_t raw_params;
  raw_params.rows   = 5;
  raw_params.cols   = 5;
  raw_params.minVal = 0;
  raw_params.maxVal = 1.0f / 25.0f;
  raw_params.type   = gpuTKType_float;

  // Generation parameters are just the image generation parameters
  gpuTKGenerateParams_t params;
  params.raw = raw_params;

  char *mask_file_name = gpuTKPath_join(dirName, "input1.raw");
  gpuTKDataset_generate(mask_file_name, gpuTKExportKind_raw, params);
  return mask_file_name;
}

float clamp(float x) {
  return std::min(std::max(x, 0.0f), 1.0f);
}

void compute(gpuTKImage_t output, gpuTKImage_t input, float *mask, int mask_rows,
             int mask_cols) {

  const int num_channels = 3;

  float *inputData  = gpuTKImage_getData(input);
  float *outputData = gpuTKImage_getData(output);

  int img_width  = gpuTKImage_getWidth(input);
  int img_height = gpuTKImage_getHeight(input);

  assert(img_width == gpuTKImage_getWidth(output));
  assert(img_height == gpuTKImage_getHeight(output));
  assert(mask_rows % 2 == 1);
  assert(mask_cols % 2 == 1);

  int mask_radius_y = mask_rows / 2;
  int mask_radius_x = mask_cols / 2;

  for (int out_y = 0; out_y < img_height; ++out_y) {
    for (int out_x = 0; out_x < img_width; ++out_x) {
      for (int c = 0; c < num_channels; ++c) { // channels
        float acc = 0;
        for (int off_y = -mask_radius_y; off_y <= mask_radius_y; ++off_y) {
          for (int off_x = -mask_radius_x; off_x <= mask_radius_x;
               ++off_x) {
            int in_y   = out_y + off_y;
            int in_x   = out_x + off_x;
            int mask_y = mask_radius_y + off_y;
            int mask_x = mask_radius_x + off_x;
            if (in_y < img_height && in_y >= 0 && in_x < img_width &&
                in_x >= 0) {
              acc +=
                  inputData[(in_y * img_width + in_x) * num_channels + c] *
                  mask[mask_y * mask_cols + mask_x];
            } else {
              acc += 0.0f;
            }
          }
        }
        // fprintf(stderr, "%f %f\n", clamp(acc));
        outputData[(out_y * img_width + out_x) * num_channels + c] =
            clamp(acc);
      }
    }
  }
}

void generate(int datasetNum, int height, int width, int minVal,
              int maxVal) {
  char *dir_name = gpuTKPath_join(base_dir, datasetNum);

  // Image generation parameters
  gpuTKPPM_GenerateParams_t ppm_params;
  ppm_params.height   = height;
  ppm_params.width    = width;
  ppm_params.channels = 3;
  ppm_params.minVal   = minVal;
  ppm_params.maxVal   = maxVal;

  // Generation parameters are just the image generation parameters
  gpuTKGenerateParams_t params;
  params.ppm = ppm_params;

  char *input_image_file_name =
      generateInput(datasetNum, dir_name, params);
  char *input_mask_file_name = generateMask(datasetNum, dir_name);

  // Import mask and image
  gpuTKImage_t inputImage = gpuTKImport(input_image_file_name);
  int mask_rows, mask_cols;
  float *mask_data =
      (float *)gpuTKImport(input_mask_file_name, &mask_rows, &mask_cols);

  // Create output image
  gpuTKImage_t outputImage = gpuTKImage_new(width, height, 3);
  compute(outputImage, inputImage, mask_data, mask_rows, mask_cols);

  // Exporto output image
  char *output_file_name = gpuTKPath_join(dir_name, "output.ppm");
  gpuTKExport(output_file_name, outputImage);

  free(input_image_file_name);
  free(input_mask_file_name);
  free(output_file_name);
}

int main(void) {
  base_dir = gpuTKPath_join(gpuTKDirectory_current(), "Convolution", "Dataset");
  generate(0, 64, 64, 0, 1);
  generate(1, 128, 64, 0, 1);
  generate(2, 64, 128, 0, 1);
  generate(3, 64, 5, 0, 1);
  generate(4, 64, 3, 0, 1);
  generate(5, 228, 128, 0, 1);
  generate(6, 28, 12, 0, 1);

  return 0;
}

Writing conv.cu


In [6]:
!wget wget https://developer.download.nvidia.com/compute/cuda/11.4.0/local_installers/cuda_11.4.0_470.42.01_linux.run
!chmod +x cuda_11.4.0_470.42.01_linux.run
!./cuda_11.4.0_470.42.01_linux.run --silent --toolkit --override


--2023-05-18 12:15:36--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2023-05-18 12:15:36--  https://developer.download.nvidia.com/compute/cuda/11.4.0/local_installers/cuda_11.4.0_470.42.01_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3773273383 (3.5G) [application/octet-stream]
Saving to: ‘cuda_11.4.0_470.42.01_linux.run.1’

cuda_11.4.0_470.42. 100%[===================>]   3.51G   236MB/s    in 15s     

2023-05-18 12:15:52 (235 MB/s) - ‘cuda_11.4.0_470.42.01_linux.run.1’ saved [3773273383/3773273383]

FINISHED --2023-05-18 12:15:52--
Total wall clock time: 15s
Downloaded: 1 files, 3.5G in 15s (235 MB/s)


In [4]:
!nvcc conv.cu

conv.cu:2:10: fatal error: libgputk/gputk.h: No such file or directory
    2 | #include "libgputk/gputk.h"
      |          ^~~~~~~~~~~~~~~~~~
compilation terminated.
